# Pedro - Short Queeze Predictor

In [7]:
# Initial imports
import os
from sec_api import InsiderTradingApi
import pandas as pd
import math
from dotenv import load_dotenv

In [8]:
# Load .env enviroment variables
load_dotenv(dotenv_path="SEC.env")

True

In [9]:
# Set SEC API key and secret
api_key = os.getenv("SEC_API_KEY")

In [10]:
def flatten_filing(filing):
    transactions = []
    base_data = {"periodOfReport": filing["periodOfReport"], 
                "issuerCik": filing["issuer"]["cik"],
                "issuerTicker": filing["issuer"]["tradingSymbol"],
                "reportingPerson": filing["reportingOwner"]["name"]
                }
    if "transactions" not in filing["nonDerivativeTable"]:
        return []
    for transaction in filing["nonDerivativeTable"]["transactions"]:
        entry = {"securityTitle": transaction["securityTitle"],
                "codingCode": transaction["coding"]["code"],
                "acquiredDisposed": transaction["amounts"]["acquiredDisposedCode"],
                "shares": transaction["amounts"]["shares"],
                "sharePrice": transaction["amounts"]["pricePerShare"],
                "total": math.ceil(transaction["amounts"]["shares"] * transaction["amounts"]["pricePerShare"]),
                "sharesOwnedFollowingTransaction": transaction["postTransactionAmounts"]["sharesOwnedFollowingTransaction"]
                }
        transactions.append({**base_data, **entry})
    return transactions

def flatten_filings(filings):
    unflattened_list = list(map(flatten_filing, filings))
    return [item for sublist in unflattened_list for item in sublist]

In [13]:
# Define your function here
def get_insider_trades(symbol, start_date, end_date, api_key):
    insiderTradingApi = InsiderTradingApi(api_key)
    query = {
        "query": {"query_string": {"query": f"issuer.tradingSymbol:{symbol} AND filedAt:{{gte:'{start_date}', lte:'{end_date}'}}"}},
        "from": "0",
        "size": "50",
        "sort": [{ "filedAt": { "order": "desc" } }]
    }
    insider_trades = insiderTradingApi.get_data(query)

    transactions = flatten_filings(insider_trades["transactions"])

    df = pd.DataFrame(transactions)

    return df

# Your list of stock symbols
symbols = ["BRK-A", "TSLA", "TPL"]

# Your start and end dates
start_date = '2023-01-01'
end_date = '2023-06-30'

# Iterate through your list of symbols and call your function
for symbol in symbols:
    df = get_insider_trades(symbol, start_date, end_date, api_key)
    print(df)

Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
